<a href="https://colab.research.google.com/github/carmeniturbe/ejercicios_adv_machine_learning/blob/main/04_Ejercicio_de_ingenier%C3%ADa_de_caracter%C3%ADsticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este ejercicio trabajarás con los datos sobre el alquiler de bicicletas compartidas.  Puedes descargar los datos aquí.

La tarea es diseñar algunas nuevas características para intentar mejorar la capacidad de un modelo para predecir el número total del alquiler de bicicletas compartidas durante una hora determinada del día.

1. Importa los datos y elimina las columnas “casual” y “registered”.  Aquellas son redundantes para tu objetivo, “count”.  



Transforma la columna de fecha y hora en un tipo de fecha hora y utilízalo para crear 3 columnas nuevas en el dataframe que contenga:

  1. Nombre del mes   2. Nombre del día de la semana   3. Hora del día

   Asegúrate que todas las 3 nuevas columnas son un tipo de datos “objeto” para que se más tarde se le aplica una codificación one-hot.  

   Elimina las columnas “datetime” y “season”.  Estas son ahora redundantes.



Las temperaturas en la columna “temp” y “atemp” son en Celsius. Utiliza .appply() para convertirlas en Fahrenheit.



Crea una nueva columna, “temp_variance” que es la diferencia entre “temp” y “atemp”. Elimina la columna “atemp”.



##Opcional:
Utiliza un modelo predictivo de tu elección e intenta predecir el “count” de usuarios de bicicletas compartidas por hora, con las características originales y el conjunto de características de ingeniería que acabas de crear.  

Recuerda eliminar las columnas “casual” y “registered” de ambas versiones antes de realizar una modelización.

¿Estas opciones de ingeniería de características mejoran tu habilidad de predecir “count”?

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Get data
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Science - Coding Dojo/Data/bikeshare_train - bikeshare_train.csv')
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [4]:
df = df.drop(columns=["casual", "registered"],axis=1)
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,1


Transformaremos la columna de fecha y hora en tipo datetime y la utilizaremos para crear 3 columnas:

1-) Nombre del mes
2-) Nombre del día de la semana
3-) Hora del día

Nos aseguraremos de que la columna sea de tipo objeto para poder aplicar one-hot despues

También eliminaremos la columnas datetime y season porque serán redundantes

In [5]:
# Convierte la columna 'datetime' en tipo datetime
df2 = df
df2['datetime'] = pd.to_datetime(df2['datetime'])

# Crearemos las columnas "Nombre del mes", "Nombre del día de la semana" y "Hora del día"
df2['Nombre del mes'] = df2['datetime'].dt.month_name()
df2['Nombre del día de la semana'] = df2['datetime'].dt.day_name()
df2['Hora del día'] = df2['datetime'].dt.strftime('%H:%M:%S')

# Cambiaremos las columnas a tipo "objeto"
df2['Nombre del mes'] = df2['Nombre del mes'].astype('object')
df2['Nombre del día de la semana'] = df2['Nombre del día de la semana'].astype('object')
df2['Hora del día'] = df2['Hora del día'].astype('object')

# Eliminaremos las columnas "datetime" y "season"
df2.drop(['datetime', 'season'], axis=1, inplace=True)

df2.head()
# Ahora, el DataFrame df contiene las tres nuevas columnas y las columnas eliminadas.

,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,Nombre del mes,Nombre del día de la semana,Hora del día
0,0,0,1,9.84,14.395,81,0.0,16,January,Saturday,00:00:00
1,0,0,1,9.02,13.635,80,0.0,40,January,Saturday,01:00:00
2,0,0,1,9.02,13.635,80,0.0,32,January,Saturday,02:00:00
3,0,0,1,9.84,14.395,75,0.0,13,January,Saturday,03:00:00
4,0,0,1,9.84,14.395,75,0.0,1,January,Saturday,04:00:00


Ahora procederemos a cambiar la columna temp y atemp de celsius a farenheit.

Luego crearemos una nueva columna llamada "temp_variance" que será la diferencia entre temp y atemp

Finalmente eliminaremos la columna "atemp"

In [6]:
# Función para convertir de Celsius a Fahrenheit
def celsius_to_fahrenheit(celsius):
    return (celsius * 9/5) + 32

# Aplicaremos la conversión a las columnas "temp" y "atemp" en df2
df2['temp'] = df2['temp'].apply(celsius_to_fahrenheit)
df2['atemp'] = df2['atemp'].apply(celsius_to_fahrenheit)

# Crearemos la columna "temp_variance" en df2
df2['temp_variance'] = df2['temp'] - df2['atemp']

# Eliminaremos la columna "atemp" en df2
df2.drop('atemp', axis=1, inplace=True)

df2.head()

,holiday,workingday,weather,temp,humidity,windspeed,count,Nombre del mes,Nombre del día de la semana,Hora del día,temp_variance
0,0,0,1,49.712,81,0.0,16,January,Saturday,00:00:00,-8.199
1,0,0,1,48.236,80,0.0,40,January,Saturday,01:00:00,-8.307
2,0,0,1,48.236,80,0.0,32,January,Saturday,02:00:00,-8.307
3,0,0,1,49.712,75,0.0,13,January,Saturday,03:00:00,-8.199
4,0,0,1,49.712,75,0.0,1,January,Saturday,04:00:00,-8.199


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   holiday                      10886 non-null  int64  
 1   workingday                   10886 non-null  int64  
 2   weather                      10886 non-null  int64  
 3   temp                         10886 non-null  float64
 4   humidity                     10886 non-null  int64  
 5   windspeed                    10886 non-null  float64
 6   count                        10886 non-null  int64  
 7   Nombre del mes               10886 non-null  object 
 8   Nombre del día de la semana  10886 non-null  object 
 9   Hora del día                 10886 non-null  object 
 10  temp_variance                10886 non-null  float64
dtypes: float64(3), int64(5), object(3)
memory usage: 935.6+ KB


Opcional:

Utiliza un modelo predictivo de tu elección e intenta predecir el “count” de usuarios de bicicletas compartidas por hora, con las características originales y el conjunto de características de ingeniería que acabas de crear.  



# Modelo de regresion lineal en Dataset Original

In [9]:
X = df.drop(columns= ['count'], axis=1)
y = df['count']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X.dtypes

holiday                          int64
workingday                       int64
weather                          int64
temp                           float64
humidity                         int64
windspeed                      float64
Nombre del mes                  object
Nombre del día de la semana     object
Hora del día                    object
temp_variance                  float64
dtype: object

In [10]:
# Selectors
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [11]:
# Imputers
freq_imputer = SimpleImputer(strategy='most_frequent')  # This won't be needed in this model
# Scaler
scaler = StandardScaler() # for numeric data
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False) # to turn categorical variables into a numerical representation

In [12]:
# Numeric pipeline
numeric_pipe = make_pipeline(freq_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('standardscaler', StandardScaler())])

In [13]:
# Categorical pipeline
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [14]:
# Tuples para Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)

# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple, remainder= 'passthrough')
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7bf0718d6c50>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7bf0718d5a80>)])

In [15]:
# fit on train
preprocessor.fit(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7bf0718d6c50>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7bf0718d5a80>)])

In [16]:
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [17]:
# checking the final values
print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'missing values in testing data')
print('\n')
print('All data in X_train_processed are', X_train_processed.dtype)
print('All data in X_test_processed are', X_test_processed.dtype)
print('\n')
print('shape of data is', X_train_processed.shape)
print('\n')
X_train_processed

0 missing values in training data
0 missing values in testing data


All data in X_train_processed are float64
All data in X_test_processed are float64


shape of data is (8164, 50)




array([[-0.17403371,  0.68088694, -0.6582388 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.17403371,  0.68088694, -0.6582388 , ...,  0.        ,
         1.        ,  0.        ],
       [-0.17403371,  0.68088694, -0.6582388 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.17403371, -1.46867261, -0.6582388 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.17403371, -1.46867261, -0.6582388 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.17403371,  0.68088694,  0.92603642, ...,  0.        ,
         0.        ,  0.        ]])

In [18]:
# Fit the Linear Regression model
lr_model_df = LinearRegression()
lr_model_df.fit(X_train_processed, y_train)

LinearRegression()

In [19]:
# predictions
# Train set predictions
train_preds = lr_model_df.predict(X_train_processed)
# Test set predictions
test_preds = lr_model_df.predict(X_test_processed)
print(train_preds)
print(test_preds)

[ 88.20117188 194.47460938  62.29296875 ... 266.48242188 227.00390625
 231.2890625 ]
[278.70117188 -26.45507812 389.85742188 ...  52.88867188 411.67578125
 171.16015625]


In [20]:
# Linear regresion R2 scores
train_score1 = lr_model_df.score(X_train_processed, y_train)
print(train_score1)

0.6375776322804906


In [21]:
#Linear regresion R2 scores
test_score1 = lr_model_df.score(X_test_processed, y_test)
print(test_score1)

0.6256758511165772


In [22]:
#RMSE
print('Training RMSE of original df:', np.sqrt(mean_squared_error(y_train, lr_model_df.predict(X_train_processed))))
print('Testing RMSE of original df:', np.sqrt(mean_squared_error(y_test, lr_model_df.predict(X_test_processed))))

Training RMSE of original df: 109.04427784490046
Testing RMSE of original df: 110.82909198503688


# Modelo de regresión lineal en Dataset con ingenieria de caracteristicas

In [23]:
X = df2.drop(columns= ['count'], axis=1)
y = df2['count']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X.dtypes

holiday                          int64
workingday                       int64
weather                          int64
temp                           float64
humidity                         int64
windspeed                      float64
Nombre del mes                  object
Nombre del día de la semana     object
Hora del día                    object
temp_variance                  float64
dtype: object

In [38]:
# Selectors
cat_selector2 = make_column_selector(dtype_include='object')
num_selector2 = make_column_selector(dtype_include='number')

In [39]:
# Imputers
freq_imputer2 = SimpleImputer(strategy='most_frequent')  # This won't be needed in this model
# Scaler
scaler2 = StandardScaler() # for numeric data
# One-hot encoder
ohe2 = OneHotEncoder(handle_unknown='ignore', sparse=False) # to turn categorical variables into a numerical representation

In [40]:
# Numeric pipeline
numeric_pipe2 = make_pipeline(freq_imputer2, scaler2)
numeric_pipe2

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('standardscaler', StandardScaler())])

In [41]:
# Categorical pipeline
categorical_pipe2 = make_pipeline(freq_imputer2, ohe2)
categorical_pipe2

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [43]:
# Tuples para Column Transformer
number_tuple2 = (numeric_pipe2, num_selector2)
category_tuple2 = (categorical_pipe2, cat_selector2)

# ColumnTransformer
preprocessor2 = make_column_transformer(number_tuple2, category_tuple2, remainder= 'passthrough')
preprocessor2

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7bf070484ee0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7bf070485240>)])

In [46]:
# fit on train
preprocessor2.fit(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7bf070484ee0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7bf070485240>)])

In [47]:
# transform train and test
X_train_processed2 = preprocessor2.transform(X_train)
X_test_processed2 = preprocessor2.transform(X_test)

In [48]:
# checking the final values
print(np.isnan(X_train_processed2).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed2).sum().sum(), 'missing values in testing data')
print('\n')
print('All data in X_train_processed are', X_train_processed2.dtype)
print('All data in X_test_processed are', X_test_processed2.dtype)
print('\n')
print('shape of data is', X_train_processed2.shape)
print('\n')
X_train_processed2

0 missing values in training data
0 missing values in testing data


All data in X_train_processed are float64
All data in X_test_processed are float64


shape of data is (8164, 50)




array([[-0.17403371,  0.68088694, -0.6582388 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.17403371,  0.68088694, -0.6582388 , ...,  0.        ,
         1.        ,  0.        ],
       [-0.17403371,  0.68088694, -0.6582388 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.17403371, -1.46867261, -0.6582388 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.17403371, -1.46867261, -0.6582388 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.17403371,  0.68088694,  0.92603642, ...,  0.        ,
         0.        ,  0.        ]])

In [49]:
# Fit the Linear Regression model
lr_model_df2 = LinearRegression()
lr_model_df2.fit(X_train_processed2, y_train)

LinearRegression()

In [50]:
# predictions
# Train set predictions
train_preds2 = lr_model_df2.predict(X_train_processed2)
# Test set predictions
test_preds2 = lr_model_df2.predict(X_test_processed2)
print(train_preds2)
print(test_preds2)

[ 88.20117188 194.47460938  62.29296875 ... 266.48242188 227.00390625
 231.2890625 ]
[278.70117188 -26.45507812 389.85742188 ...  52.88867188 411.67578125
 171.16015625]


In [51]:
# Linear regresion R2 scores
train_score2 = lr_model_df2.score(X_train_processed2, y_train)
print(train_score2)

0.6375776322804906


In [52]:
#Linear regresion R2 scores
test_score2 = lr_model_df2.score(X_test_processed2, y_test)
print(test_score2)

0.6256758511165772


In [53]:
#RMSE
print('Training RMSE of processed df:', np.sqrt(mean_squared_error(y_train, lr_model_df2.predict(X_train_processed2))))
print('Testing RMSE of  processed df:', np.sqrt(mean_squared_error(y_test, lr_model_df2.predict(X_test_processed2))))

Training RMSE of processed df: 109.04427784490046
Testing RMSE of  processed df: 110.82909198503688


¿Estas opciones de ingeniería de características mejoran tu habilidad de predecir “count”?

No hay ninguna diferencia perceptible entre el modelo con ingenieria y el modelo sin ingenieria.